In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Activation
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn import model_selection
from sklearn import metrics
import time
import tensorflow as tf
import shap
import numpy as np

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
df = pd.read_csv('X-IIoTID dataset_processed_lime.csv')
columns = df.columns.tolist()

In [3]:
#read kdd_processed_bin_top15.csv
df_bin = pd.read_csv('X-IIoTID dataset_processed_top15.csv')

# get the names of the columns in a list
columns_bin = df_bin.columns.tolist()

In [4]:
#combine the two lists of columns without duplicates
columns = list(set(columns + columns_bin))
print(columns)
#print length of columns
print(len(columns))

['Service_simple_service_discovery', 'is_syn_only_False', 'Scr_port', 'Service_dns', 'Service_private', 'Scr_bytes_ratio', 'Protocol_udp', 'Service_netbios-ns', 'OSSEC_alert_level', 'Process_activity_1', 'Service_websocket', 'Des_bytes_ratio', 'Service_imap', 'Conn_state', 'Service_echo', 'is_pure_ack_True', 'Service_mysql', 'Service_https', 'is_syn_only_True', 'Service_other', 'class3', 'Avg_num_cswch/s', 'Service_modbus', 'Unnamed: 0', 'read_write_physical.process_0', 'is_with_payload_True', 'FIN or RST_False']
27


In [5]:
data = pd.read_csv('X-IIoTID dataset_processed.csv')

In [6]:
data = data.drop(data.columns.difference(columns), axis=1)
data

,Scr_port,Conn_state,Scr_bytes_ratio,Des_bytes_ratio,Avg_num_cswch/s,OSSEC_alert_level,class3,Protocol_udp,Service_dns,Service_echo,...,Service_private,Service_simple_service_discovery,Service_websocket,is_syn_only_False,is_syn_only_True,is_pure_ack_True,is_with_payload_True,FIN or RST_False,Process_activity_1,read_write_physical.process_0
0,0.751923,1.0,0.300414,0.699586,0.167705,0.5,0,False,False,False,...,False,False,False,False,True,True,True,False,False,True
1,0.606818,1.0,1.000000,0.000000,0.110795,0.0,1,True,True,False,...,False,False,False,True,False,False,True,True,False,True
2,0.901041,1.0,0.500000,0.500000,0.112701,0.0,1,True,True,False,...,False,False,False,True,False,False,True,True,False,True
3,0.579305,1.0,0.674254,0.325746,0.116451,0.0,1,False,False,False,...,False,False,True,False,True,True,True,False,True,False
4,0.770601,1.0,0.442110,0.557890,0.101892,0.0,1,False,False,False,...,False,False,False,False,True,True,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596033,0.782656,1.0,0.392613,0.607387,0.161770,0.5,0,False,False,False,...,False,False,False,False,True,True,True,False,False,True
596034,0.700574,1.0,0.568421,0.431579,0.105984,0.0,1,False,False,False,...,False,False,True,False,True,True,False,False,False,True
596035,0.812565,0.0,0.048713,0.951287,0.087689,0.0,1,False,False,False,...,False,False,False,False,True,True,True,False,False,False
596036,0.678264,0.0,0.523810,0.476190,0.092813,0.0,0,False,False,False,...,False,False,False,False,True,False,False,False,False,True


In [7]:
# train an lstm model on the data
X = data.drop(['class3'], axis=1)
Y = data['class3']

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)

# convert Y to int
Y_train = Y_train.astype(int)
Y_test = Y_test.astype(int)

# reshape the data for LSTM
X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

# convert all to numpy arrays
X_train = np.asarray(X_train).astype(np.float32)
# Y_train = np.asarray(Y_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)
# Y_test = np.asarray(Y_test).astype(np.float32)

# define the model and add recurrent dropout
model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True, activation = 'relu'))
model.add(Dropout(0.1))
model.add(LSTM(64, return_sequences=True, activation = 'relu'))
model.add(Dropout(0.1))
model.add(LSTM(64, activation = 'relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# train the model
start = time.time()
history = model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_data=(X_test, Y_test), shuffle=True)
end = time.time()
print("Time taken to train the model: ", end-start)

Epoch 1/10
14901/14901 [==============================] - 29s 2ms/step - loss: 0.1122 - accuracy: 0.9653 - val_loss: 0.0756 - val_accuracy: 0.9726
Epoch 2/10
14901/14901 [==============================] - 28s 2ms/step - loss: 0.0795 - accuracy: 0.9733 - val_loss: 0.0719 - val_accuracy: 0.9766
Epoch 3/10
14901/14901 [==============================] - 31s 2ms/step - loss: 0.0746 - accuracy: 0.9749 - val_loss: 0.0681 - val_accuracy: 0.9775
Epoch 4/10
14901/14901 [==============================] - 32s 2ms/step - loss: 0.0724 - accuracy: 0.9755 - val_loss: 0.0706 - val_accuracy: 0.9750
Epoch 5/10
14901/14901 [==============================] - 30s 2ms/step - loss: 0.0709 - accuracy: 0.9756 - val_loss: 0.0660 - val_accuracy: 0.9766
Epoch 6/10
14901/14901 [==============================] - 29s 2ms/step - loss: 0.0692 - accuracy: 0.9759 - val_loss: 0.0635 - val_accuracy: 0.9777
Epoch 7/10
14901/14901 [==============================] - 28s 2ms/step - loss: 0.0686 - accuracy: 0.9759 - val_loss: 0

5-FOLD CROSS VALIDATION

In [ ]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits=5, shuffle=True)
X = data.drop(['class3'], axis=1)
Y = data['class3']

# reshape the data for LSTM
X_fit = X.values.reshape((X.shape[0], 1, X.shape[1]))
X_fit = np.asarray(X_fit).astype(np.float32)

# define the model
model = Sequential()
model.add(LSTM(64, input_shape=(X_fit.shape[1], X_fit.shape[2]), return_sequences=True, activation = 'relu'))
model.add(Dropout(0.1))
model.add(LSTM(64, return_sequences=True, activation = 'relu'))
model.add(Dropout(0.1))
model.add(LSTM(64, activation = 'relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


accuracies = []

for train_idx, test_idx in kfold.split(X_fit):
    model.fit(X_fit[train_idx], Y[train_idx], epochs=10, batch_size=32, verbose=0)
    accuracy = model.evaluate(X_fit[test_idx], Y[test_idx], verbose=0)[1] 
    accuracies.append(accuracy)

print(accuracies)
#mean accuracy
print("Mean- ",np.mean(accuracies))

In [8]:
start = time.time()
y_pred = model.predict(X_test)
end = time.time()
y_pred = (y_pred > 0.5)
print('Classification Report')
print(metrics.classification_report(Y_test, y_pred, digits=4))
print('Accuracy:')
print(metrics.accuracy_score(Y_test, y_pred))
print("Time taken to predict: ", end-start)

3726/3726 [==============================] - 2s 545us/step
Classification Report
              precision    recall  f1-score   support

           0     0.9937    0.9552    0.9741     51897
           1     0.9665    0.9953    0.9807     67311

    accuracy                         0.9779    119208
   macro avg     0.9801    0.9753    0.9774    119208
weighted avg     0.9783    0.9779    0.9778    119208

Accuracy:
0.9778873901080465
Time taken to predict:  2.83296275138855
